In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Liver_cirrhosis"
cohort = "GSE185529"

# Input paths
in_trait_dir = "../../input/GEO/Liver_cirrhosis"
in_cohort_dir = "../../input/GEO/Liver_cirrhosis/GSE185529"

# Output paths
out_data_file = "../../output/preprocess/Liver_cirrhosis/GSE185529.csv"
out_gene_data_file = "../../output/preprocess/Liver_cirrhosis/gene_data/GSE185529.csv"
out_clinical_data_file = "../../output/preprocess/Liver_cirrhosis/clinical_data/GSE185529.csv"
json_path = "../../output/preprocess/Liver_cirrhosis/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import numpy as np
import os

# 1. Gene Expression Data Availability
# Based on the background information, this appears to focus on genes (BNC2) regulated in myofibroblasts
# and liver fibrosis, but we need to be cautious as it's a SuperSeries composed of SubSeries.
# Without seeing the actual gene data, we'll assume it likely contains gene expression data.
is_gene_available = True  # Set to True assuming gene expression data is available

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics dictionary, we only see 'treatment' information
# There is no explicit trait (liver cirrhosis), age, or gender information
trait_row = None  # No explicit liver cirrhosis trait information
age_row = None    # No age information
gender_row = None # No gender information

# 2.2 Data Type Conversion Functions
# Since we don't have trait, age, or gender data, these functions are placeholders
def convert_trait(value):
    if pd.isna(value) or value is None:
        return None
    
    value_str = str(value)
    if ':' in value_str:
        value_str = value_str.split(':', 1)[1].strip()
    
    # For liver cirrhosis, typically would convert to binary (0: no cirrhosis, 1: cirrhosis)
    # But since we don't have the data, this is just a placeholder
    return None

def convert_age(value):
    if pd.isna(value) or value is None:
        return None
    
    value_str = str(value)
    if ':' in value_str:
        value_str = value_str.split(':', 1)[1].strip()
    
    # Would convert to continuous numeric value
    # But since we don't have the data, this is just a placeholder
    return None

def convert_gender(value):
    if pd.isna(value) or value is None:
        return None
    
    value_str = str(value)
    if ':' in value_str:
        value_str = value_str.split(':', 1)[1].strip().lower()
    
    # Would convert to binary (0: female, 1: male)
    # But since we don't have the data, this is just a placeholder
    return None

# 3. Save Metadata
# Determine trait data availability based on whether trait_row is None
is_trait_available = trait_row is not None

# Save the initial filtering results
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None (clinical data not available for our needed traits)


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


### Step 4: Gene Identifier Review

In [ ]:
# Examining the gene identifiers in the data
# These identifiers (like '2824546_st') are not standard human gene symbols
# They appear to be probe IDs from a microarray platform (GPL17586)
# These would need to be mapped to standard gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")


### Step 6: Gene Identifier Mapping

In [ ]:
# Extract the probe_id and gene_symbols from gene_annotation
# Looking at the gene_data IDs (2824546_st) and the gene_annotation data
# Need to map between probe IDs and gene symbols

# Create a mapping between the probe IDs in gene_data and gene symbols
# The ID column in gene_annotation needs to be linked to the probe IDs in gene_data
# We need to explore how the IDs match up

# Extract the ID suffix pattern from gene_data
id_pattern = gene_data.index[0].split('_')[1]  # Should be "st"
print(f"ID suffix pattern in gene_data: {id_pattern}")

# Check how the IDs in gene_annotation can be matched to gene_data
sample_gene_data_ids = gene_data.index[:5].tolist()
print(f"Sample gene data IDs: {sample_gene_data_ids}")

# Looking at "gene_assignment" column which contains gene symbols
# First check if gene_assignment actually contains mappable data (not all "---")
gene_mappings_available = gene_annotation['gene_assignment'].apply(lambda x: x != '---').sum()
print(f"Number of rows with gene mappings: {gene_mappings_available} out of {len(gene_annotation)}")

# 1. Create the gene mapping dataframe
# We'll map from the ID in gene_annotation to the gene symbols in gene_assignment
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')
print(f"Gene mapping shape: {mapping_df.shape}")
print("First few mappings:")
print(mapping_df.head())

# 2. Apply the gene mapping to convert probe-level measurements to gene expression
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Mapped gene data shape: {gene_data.shape}")
print("First few genes:")
print(gene_data.index[:10])

# Save the gene data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Check if trait_row is None (indicating no clinical data is available)
if trait_row is None:
    print("No clinical data available for this dataset, skipping clinical data processing.")
    
    # Validate and save cohort information with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use this data without clinical features
        df=pd.DataFrame(),  # Empty DataFrame since we have no linked data
        note="Dataset contains gene expression data from cell lines with HCV infection, which is not appropriate for liver cirrhosis trait analysis."
    )
    
    print("Dataset is not usable for liver cirrhosis analysis due to lack of clinical data. No linked data file saved.")
else:
    # If clinical data is available, proceed with the linking and processing
    # 2. Link the clinical and genetic data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty after missing value handling")

    # 5. Validate and save cohort information
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note="Dataset contains gene expression data for liver fibrosis progression, which is relevant to liver cirrhosis research."
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")